---

Created for [learn-investments.rice-business.org](https://learn-investments.rice-business.org)
    
By [Kerry Back](https://kerryback.com) and [Kevin Crotty](https://kpcrotty.github.io/)
    
Jones Graduate School of Business, Rice University

---


# EXAMPLE DATA

In [1]:
TICKER    = 'FCNTX'
BENCHMARK = 'SPY'

#### Some potential benchmark ETFs: 
# 'SPY = S&P 500 ETF', 
# 'IVE = S&P 500 Value ETF', 
# 'IVW = S&P 500 Growth ETF', 
# 'IWB = Russell 1000 ETF', 
# 'IWD = Russell 1000 Value ETF', 
# 'IWF = Russell 1000 Growth ETF', 
# 'IWM = Russell 2000 ETF', 
# 'IWN = Russell 2000 Value ETF', 
# 'IWO = Russell 2000 Growth ETF', 
# 'IWV = Russell 3000 ETF'

# Date Range (input a year)
start_yr = 2000
stop_yr  = 2023

# GET DATA

In [2]:
import numpy as np
import pandas as pd
from pandas_datareader import DataReader as pdr
import statsmodels.api as sm
from scipy.stats import ttest_1samp as ttest
import yfinance as yf
import plotly.graph_objects as go
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
# fund returns
TICKER = TICKER.upper()
rets = yf.download(TICKER,start='1970-01-01', progress=False)
rets = rets['Adj Close'].resample('M').last().pct_change().dropna()
rets.name = TICKER
rets.index = rets.index.to_period('M')

In [4]:
# benchmark returns
benchmark = yf.download(BENCHMARK,start='1970-01-01', progress=False)
benchmark = benchmark['Adj Close'].resample('M').last().pct_change().dropna()
benchmark.name = BENCHMARK
benchmark.index = benchmark.index.to_period('M')

In [5]:
# ff_monthly data
ff3    = pdr('F-F_Research_Data_Factors','famafrench', start=1900)[0]/100
ff5    = pdr('F-F_Research_Data_5_Factors_2x3','famafrench', start=1900)[0]/100
Mom    = pdr('F-F_Momentum_Factor','famafrench', start=1900)[0]/100
ST_Rev = pdr('F-F_ST_Reversal_Factor','famafrench', start=1900)[0]/100
LT_Rev = pdr('F-F_LT_Reversal_Factor','famafrench', start=1900)[0]/100
ff48   = pdr("48_Industry_Portfolios", "famafrench", start=1900)[0]/100

facts = pd.concat((ff5, Mom, ST_Rev, LT_Rev), axis=1).dropna()

In [6]:
# combine data
df = pd.concat((rets, benchmark, facts), axis=1).dropna()
df[TICKER + '-RF']   = df[TICKER] - df.RF
df[BENCHMARK +'-RF'] = df[BENCHMARK] - df.RF

In [7]:
# Subset for start and end dates
start = str(start_yr)+'-01'
stop  = str(stop_yr)+'-12'
df = df.loc[start:stop]

# SIMPLE BENCHMARK ADJUSTMENT
- active return  = return - benchmark return
- tracking error = sd(active return)
- information ratio = mean(active return)/sd(active return) 

In [8]:
xrets = 100 * 100 * (df[TICKER] - df[BENCHMARK])    # expressed in basis points per month

# Statistics of interest
xrets_mn   = xrets.mean()
xrets_sd   = xrets.std()
xrets_info = xrets_mn/xrets_sd
xrets_t, xrets_p = ttest(xrets,0)
xrets_stderr   = xrets_mn/xrets_t

print(f'Simple excess-of-benchmark performance for {TICKER} relative to benchmark {BENCHMARK}:')
print(f'     mean excess return (bps):\t\t{xrets_mn:.2f}')
print(f'     tracking error (bps):\t\t{xrets_sd:.2f}')
print(f'     stderr(mean excess return): \t{xrets_stderr:.2f}')
print(f'     t-stat(mean excess return): \t{xrets_t:.2f}')
print(f'     p-value(mean excess return):\t{xrets_p:.2f}')
print(f'     information ratio:\t\t\t{xrets_info:.2%}')

Simple excess-of-benchmark performance for FCNTX relative to benchmark SPY:
     mean excess return (bps):		18.05
     tracking error (bps):		194.92
     stderr(mean excess return): 	11.59
     t-stat(mean excess return): 	1.56
     p-value(mean excess return):	0.12
     information ratio:			9.26%


In [9]:
# Cumulative returns
contribs = pd.DataFrame(dtype=float, index=df.index, columns=['Active', 'Benchmark'])
contribs['Benchmark'] = df[BENCHMARK]
contribs['Active'] = df[TICKER] - df[BENCHMARK]
cum = (1 + contribs).cumprod()

fig = go.Figure()
trace= go.Scatter(x=cum.index.astype(str), y=cum.Active, name='Active', hovertemplate="Active: %{y:.2f}<extra></extra>")
fig.add_trace(trace)
trace= go.Scatter(x=cum.index.astype(str), y=cum.Benchmark, name='Benchmark', hovertemplate="Benchmark: %{y:.2f}<extra></extra>")
fig.add_trace(trace)
fig.update_yaxes(title='Compound Return')
fig.update_xaxes(title="Date")
fig.update_layout(hovermode='x unified')
fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
fig.show()


# BETA-ADJUSTED BENCMARK
- active return  = return - (risk-free + beta * (benchmark - risk-free))
- tracking error = sd(active return)
- information ratio = mean(active return)/sd(active return) 

In [10]:
y = df[TICKER + '-RF']
X = sm.add_constant(df[BENCHMARK +'-RF'])
results = sm.OLS(y,X).fit()

# Statistics of interest
xrets_mn = 100*100*results.params['const']
xrets_sd = 100*100*np.sqrt(results.mse_resid)
xrets_t  = results.tvalues['const']
xrets_p  = results.pvalues['const']
xrets_stderr = 100*100*results.bse['const']
xrets_info   =  xrets_mn/xrets_sd
beta = results.params[BENCHMARK +'-RF']
beta_se = results.bse[BENCHMARK +'-RF']

print(f'Beta-adjusted benchmark performance for {TICKER} relative to benchmark {BENCHMARK}:')
print(f'     alpha (bps):\t{xrets_mn:.2f}')
print(f'     stderr(alpha): \t{xrets_stderr:.2f}')
print(f'     t-stat(alpha): \t{xrets_t:.2f}')
print(f'     p-value(alpha):\t{xrets_p:.2f}')
print(f'     beta:\t\t{beta:.2f}')
print(f'     stderr(beta):\t{beta_se:.2f}')
print(f'     information ratio:\t{xrets_info:.2%}')

Beta-adjusted benchmark performance for FCNTX relative to benchmark SPY:
     alpha (bps):	25.65
     stderr(alpha): 	11.03
     t-stat(alpha): 	2.33
     p-value(alpha):	0.02
     beta:		0.86
     stderr(beta):	0.02
     information ratio:	13.93%


In [11]:
# Statistics of interest
xrets_mn = 100*100*results.params['const']
xrets_sd = 100*100*np.sqrt(results.mse_resid)
xrets_t  = results.tvalues['const']
xrets_p  = results.pvalues['const']
xrets_stderr = 100*100*results.bse['const']
xrets_info   =  xrets_mn/xrets_sd
beta = results.params[BENCHMARK +'-RF']
beta_se = results.bse[BENCHMARK +'-RF']

print(f'Beta-adjusted benchmark performance for {TICKER} relative to benchmark {BENCHMARK}:')
print(f'     alpha (bps):\t{xrets_mn:.2f}')
print(f'     stderr(alpha): \t{xrets_stderr:.2f}')
print(f'     t-stat(alpha): \t{xrets_t:.2f}')
print(f'     p-value(alpha):\t{xrets_p:.2f}')
print(f'     beta:\t\t{beta:.2f}')
print(f'     stderr(beta):\t{beta_se:.2f}')
print(f'     information ratio:\t{xrets_info:.2%}')

Beta-adjusted benchmark performance for FCNTX relative to benchmark SPY:
     alpha (bps):	25.65
     stderr(alpha): 	11.03
     t-stat(alpha): 	2.33
     p-value(alpha):	0.02
     beta:		0.86
     stderr(beta):	0.02
     information ratio:	13.93%


In [12]:
# Cumulative returns
contribs = pd.DataFrame(dtype=float, index=df.index, columns=['Active', 'Benchmark'])
contribs['Benchmark'] = beta*df[BENCHMARK] + (1-beta)*df.RF
contribs['Active'] = df[TICKER] - contribs['Benchmark']
cum = (1 + contribs).cumprod()

fig = go.Figure()
trace= go.Scatter(x=cum.index.astype(str), y=cum.Active, name='Active', hovertemplate="Active: %{y:.2f}<extra></extra>")
fig.add_trace(trace)
trace= go.Scatter(x=cum.index.astype(str), y=cum.Benchmark, name='Benchmark', hovertemplate="Benchmark: %{y:.2f}<extra></extra>")
fig.add_trace(trace)
fig.update_yaxes(title='Compound Return')
fig.update_xaxes(title="Date")
fig.update_layout(hovermode='x unified')
fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
fig.show()

In [13]:
# Benchmark model fit
import plotly.express as px

df2 = df.copy()
df2 = df2.reset_index()
df2['Date'] = df2.Date.astype(str)
fig = px.scatter(
    df2,
    x=BENCHMARK +'-RF',
    y=TICKER +'-RF',
    trendline="ols",
    hover_data={x: False for x in df2.columns.to_list()},
    hover_name="Date",
    )
fig.layout.xaxis["title"] = "Monthly Benchmark Excess Return"
fig.layout.yaxis["title"] = "Monthly Excess Return"
fig.update_traces(
        marker=dict(size=12, line=dict(width=2, color="DarkSlateGrey")),
        selector=dict(mode="markers"),
)
fig.update_yaxes(tickformat=".0%")
fig.update_xaxes(tickformat=".0%")

# MULTI-FACTOR BENCHMARK

In [14]:
factors = facts.drop(columns='RF').columns.to_list()
result = sm.OLS(df[TICKER +'-RF'], sm.add_constant(df[factors])).fit()
ff_r2 = result.rsquared
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:               FCNTX-RF   R-squared:                       0.912
Model:                            OLS   Adj. R-squared:                  0.910
Method:                 Least Squares   F-statistic:                     355.5
Date:                Wed, 04 Oct 2023   Prob (F-statistic):          6.87e-140
Time:                        15:18:56   Log-Likelihood:                 837.82
No. Observations:                 283   AIC:                            -1658.
Df Residuals:                     274   BIC:                            -1625.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0012      0.001      1.504      0.1

In [15]:
contribs = pd.DataFrame(dtype=float, index=df.index, columns=['Active']+factors)
for col in contribs.columns:
    if col not in ['Active']:
        contribs[col] = df[col] * result.params[col]
contribs['Active'] = df[TICKER+'-RF'] - contribs[factors].sum(axis=1)
contribs


cum = (1 + contribs).cumprod()
cum = cum.reset_index()
cum.Date = cum.Date.astype(str)

fig = go.Figure()
cols = [x for x in cum.columns if x!='Date']
for source in cols:
    # Exclude market excess return for ease of viewing:
    if source != 'Mkt-RF':
        trace = go.Scatter(x=cum.Date, y=cum[source],
                            hovertemplate=source + ": %{y:.2f}<extra></extra>",
                            name=source)
        fig.add_trace(trace)

fig.update_yaxes(title='Compound Return')
fig.update_xaxes(title="Date")
fig.update_layout(hovermode='x unified')
fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))